# Demo


This shows using Python to read data from the GitHub search
 API and writing it to the blob storage backed mount point



In [1]:
import json
import requests

req = requests.get("https://api.github.com/search/repositories?q=tetris&sort=stars&order=desc")

if not req.ok:
    raise "Unable to download data"

with open("/media/polydata/data/github-tetris.json", "w") as f:
    json.dump(req.json(), f)

Next we take the downloaded data back from the file system, read it into a Spark DataFrame, and perform the following:


* Explode out the repo data<br>
* Filter to only the few columns we're interested in
* Cast the numeric and timestamp values into their correct data type


*Note:* Polynote has already created the SparkSession as `spark`, and has imported the functions and implicits.




In [2]:
import org.apache.spark.sql.types.{TimestampType, IntegerType}

val df = spark.read.json("/media/polydata/data/github-tetris.json")
    .withColumn("items", explode($"items"))
    .select("items.*")
    .select("full_name", "description", "fork", "language", "open_issues", "updated_at", "watchers")
    .withColumn("open_issues", $"open_issues".cast(IntegerType))
    .withColumn("watchers", $"watchers".cast(IntegerType))
    .withColumn("updated_at", $"updated_at".cast(TimestampType))

Now we perform a simple aggregation to get a view of the number of projects by language type, and the number of projects, issues, and watchers for each language.

In [4]:
val aggregatedDF = df.groupBy($"language")
    .agg(
        count($"full_name").alias("num_projects"), 
        sum($"open_issues").alias("total_open_issues"), 
        sum("watchers").alias("total_watchers"))
    .orderBy($"total_watchers".desc)

aggregatedDF.show()

+----------+------------+-----------------+--------------+
|  language|num_projects|total_open_issues|total_watchers|
+----------+------------+-----------------+--------------+
|JavaScript|          10|               56|         13274|
|     Swift|           2|                5|          1633|
|    Python|           4|                3|          1286|
|       C++|           3|               10|          1272|
|  Assembly|           2|                2|           890|
|      Dart|           1|                0|           840|
|   Clojure|           2|               17|           581|
|   Haskell|           1|                0|           422|
|       Lua|           1|                2|           413|
|      HTML|           1|               17|           388|
|     Shell|           1|                2|           344|
|      Java|           1|                0|           326|
|        Go|           1|                0|           217|
+----------+------------+-----------------+-------------

Now lets take the aggregated data and collect it into a pandas dataframe, and use Matplotlib (note that this is added as a pre-requisite in the "Configuration & dependencies" section at the top) to graphically show the number of projects and watchers as a bar chart.

In [8]:
pandasDF = aggregatedDF.toPandas()

In [7]:
import matplotlib.pyplot as plt

fig = plt.figure()

ax = fig.add_subplot(111)
ax2 = ax.twinx()

ax1 = pandasDF.plot.bar(x="language", 
                        y="num_projects",
                        ax=ax, 
                        position=0, 
                        color="blue", 
                        width=0.4, 
                        rot=90, 
                        legend=False)

ax2 = pandasDF.plot.bar(x="language", 
                        y="total_watchers", 
                        ax=ax2, 
                        position=1, 
                        color="orange", 
                        width=0.4, 
                        legend=False)

ax.set_ylabel('Number Projects')
ax2.set_ylabel('Total Watchers')

fig.legend(labels=('No. Projects','Total Watchers'), loc='upper center')

plt.tight_layout()
plt.show()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (https://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- JavaScript -->
 
 
 <path d="M 34.28125 27.484375 
Q 23.390625 27.484375 19.1875 25 
Q 14.984375 22.515625 14.984375 16.5 
Q 14.984375 11.71875 18.140625 8.90625 
Q 21.296875 6.109375 26.703125 6.109375 
Q 34.1875 6.109375 38.703125 11.40625 
Q 43.21875 16.703125 43.21875 25.484375 
L 43.21875 27.484375 
z
M 52.203125 31.203125 
L 52.203125 0 
L 43.21875 0 
L 43.21875 8.296875 
Q 40.140625 3.328125 35.546875 0.953125 
Q 30.953125 -1.421875 24.3125 -1.421875 
Q 15.921875 -1.421875 10.953125 3.296875 
Q 6 8.015625 6 15.921875 
Q 6 25.140625 12.171875 29.828125 
Q 18.359375 34.515625 30.609375 34.515625 
L 43.21875 34.515625 
L 43.21875 35.40625 
Q 43.21875 41.609375 39.140625 45 
Q 35.0625 48.390625 27.6875 48.390625 
Q 23 48.390625 18.546875 47.265625 
Q 14.109375 46.140625 10.015625 43.890625 
L 10.015625 52.203125 
Q 14.9375 54.109375 19.578125 55.046875 
Q 24.21875 56 28.609375 56 
Q 40.484375 56 46.34375 49.84375 
Q 52.203125 43.703125 52.203125 31.203125 
z
" id="DejaVuSans-97"/>
 
 <path d="M 53.515625 70.515625 
L 53.515625 60.890625 
Q 47.90625 63.578125 42.921875 64.890625 
Q 37.9375 66.21875 33.296875 66.21875 
Q 25.25 66.21875 20.875 63.09375 
Q 16.5 59.96875 16.5 54.203125 
Q 16.5 49.359375 19.40625 46.890625 
Q 22.3125 44.4375 30.421875 42.921875 
L 36.375 41.703125 
Q 47.40625 39.59375 52.65625 34.296875 
Q 57.90625 29 57.90625 20.125 
Q 57.90625 9.515625 50.796875 4.046875 
Q 43.703125 -1.421875 29.984375 -1.421875 
Q 24.8125 -1.421875 18.96875 -0.25 
Q 13.140625 0.921875 6.890625 3.21875 
L 6.890625 13.375 
Q 12.890625 10.015625 18.65625 8.296875 
Q 24.421875 6.59375 29.984375 6.59375 
Q 38.421875 6.59375 43.015625 9.90625 
Q 47.609375 13.234375 47.609375 19.390625 
Q 47.609375 24.75 44.3125 27.78125 
Q 41.015625 30.8125 33.5 32.328125 
L 27.484375 33.5 
Q 16.453125 35.6875 11.515625 40.375 
Q 6.59375 45.0625 6.59375 53.421875 
Q 6.59375 63.09375 13.40625 68.65625 
Q 20.21875 74.21875 32.171875 74.21875 
Q 37.3125 74.21875 42.625 73.28125 
Q 47.953125 72.359375 53.515625 70.515625 
z
" id="DejaVuSans-83"/>
 <path d="M 48.78125 52.59375 
L 48.78125 44.1875 
Q 44.96875 46.296875 41.140625 47.34375 
Q 37.3125 48.390625 33.40625 48.390625 
Q 24.65625 48.390625 19.8125 42.84375 
Q 14.984375 37.3125 14.984375 27.296875 
Q 14.984375 17.28125 19.8125 11.734375 
Q 24.65625 6.203125 33.40625 6.203125 
Q 37.3125 6.203125 41.140625 7.25 
Q 44.96875 8.296875 48.78125 10.40625 
L 48.78125 2.09375 
Q 45.015625 0.34375 40.984375 -0.53125 
Q 36.96875 -1.421875 32.421875 -1.421875 
Q 20.0625 -1.421875 12.78125 6.34375 
Q 5.515625 14.109375 5.515625 27.296875 
Q 5.515625 40.671875 12.859375 48.328125 
Q 20.21875 56 33.015625 56 
Q 37.15625 56 41.109375 55.140625 
Q 45.0625 54.296875 48.78125 52.59375 
z
" id="DejaVuSans-99"/>
 
 
 <path d="M 18.109375 8.203125 
L 18.109375 -20.796875 
L 9.078125 -20.796875 
L 9.078125 54.6875 
L 18.109375 54.6875 
L 18.109375 46.390625 
Q 20.953125 51.265625 25.265625 53.625 
Q 29.59375 56 35.59375 56 
Q 45.5625 56 51.78125 48.09375 
Q 58.015625 40.1875 58.015625 27.296875 
Q 58.015625 14.40625 51.78125 6.484375 
Q 45.5625 -1.421875 35.59375 -1.421875 
Q 29.59375 -1.421875 25.265625 0.953125 
Q 20.953125 3.328125 18.109375 8.203125 
z
M 48.6875 27.296875 
Q 48.6875 37.203125 44.609375 42.84375 
Q 40.53125 48.484375 33.40625 48.484375 
Q 26.265625 48.484375 22.1875 42.84375 
Q 18.109375 37.203125 18.109375 27.296875 
Q 18.109375 17.390625 22.1875 11.75 
Q 26.265625 6.109375 33.40625 6.109375 
Q 40.53125 6.109375 44.609375 11.75 
Q 48.6875 17.390625 48.6875 27.296875 
z
" id="DejaVuSans-112"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- Swift -->
 
 
 
 